In [9]:
import os
import json
import numpy as np
from qiskit import transpile
from disqco.circuits.cp_fraction import cp_fraction
from disqco.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from disqco.parti.FM.FM_main import *
from disqco.parti.FM.multilevel_FM import *

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_MLFM-R_CP_large_2-4part_new.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_MLFM-R_CP_large_2-4part_new.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(112, 257, 16)

num_partitions_list = [2,4]

for num_partitions in num_partitions_list:
    for i, num_qubits in enumerate(sizes):

        # Create an All-to-All network
        qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
        network = QuantumNetwork(qpu_info)
        
        iteration_data = []
        for iteration in range(5):
            
            # -------------------------
            # 1. Define circuit
            # -------------------------

            circuit = cp_fraction(num_qubits,num_qubits,fraction=0.5)

            circuit = transpile(circuit, basis_gates = ['u','cp'])
            base_graph = QuantumCircuitHyperGraph(circuit,group_gates=True)
            depth = base_graph.depth
            initial_assignment = set_initial_partitions(network,num_qubits, depth)
            
            # -------------------------
            # 2. MLFM-R
            # -------------------------
            num_levels = int(np.ceil(np.log2(num_qubits)))
            
            assignment_list_r, cost_list_r, time_list_r = MLFM_recursive(base_graph,
                                        initial_assignment,  
                                        qpu_info, 
                                        limit = num_qubits, 
                                        pass_list= [10]*(num_levels+1), 
                                        stochastic=True, 
                                        lock_nodes=False,
                                        log = False,
                                        add_initial = False,
                                        costs = None)
            
            total_time_r = sum(time_list_r)
            min_cost_r = min(cost_list_r)

            
            # -------------------------
            # 3. Store iteration-level results
            # -------------------------
            result_entry = {
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "iteration": iteration,
                "r_cost":  min_cost_r,
                "time_r": total_time_r,
            }
            
            detailed_results.append(result_entry)
            iteration_data.append(result_entry)
            
            with open(detailed_filename, "w") as f:
                json.dump(detailed_results, f, indent=2)
        # -------------------------
        # 4. Aggregate results
        r_cost_list = [x["r_cost"] for x in iteration_data]
        r_time_list = [x["time_r"] for x in iteration_data]
        

        mean_r_cost = float(np.mean(r_cost_list))
        mean_r_time = float(np.mean(r_time_list))
        
        print("=============================================")
        print(f"Finished 5 iterations for:")
        print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}")
        print("Mean Costs:")

        print(f"  Recursive:{mean_r_cost:.3f}")
        print("Mean Times (s):")

        print(f"  Recursive:{mean_r_time:.3f}")
        print("=============================================")
        
        mean_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "mean_r_cost": mean_r_cost,
            "mean_r_time": mean_r_time,
        }
        
        mean_results.append(mean_entry)
        
        with open(means_filename, "w") as f:
            json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Finished 5 iterations for:
  # Qubits: 112, # Partitions: 2
Mean Costs:
  Recursive:638.400
Mean Times (s):
  Recursive:3.434
Finished 5 iterations for:
  # Qubits: 128, # Partitions: 2
Mean Costs:
  Recursive:849.000
Mean Times (s):
  Recursive:4.950
Finished 5 iterations for:
  # Qubits: 144, # Partitions: 2
Mean Costs:
  Recursive:1087.000
Mean Times (s):
  Recursive:7.126
Finished 5 iterations for:
  # Qubits: 160, # Partitions: 2
Mean Costs:
  Recursive:1330.200
Mean Times (s):
  Recursive:9.128
Finished 5 iterations for:
  # Qubits: 176, # Partitions: 2
Mean Costs:
  Recursive:1598.200
Mean Times (s):
  Recursive:11.936
Finished 5 iterations for:
  # Qubits: 192, # Partitions: 2
Mean Costs:
  Recursive:1932.000
Mean Times (s):
  Recursive:14.898
Finished 5 iterations for:
  # Qubits: 208, # Partitions: 2
Mean Costs:
  Recursive:2262.400
Mean Times (s):
  Recursive:18.689
Finished 5 iterations for:
  # Qubits: 224, # Partitions: 2
Mean Costs:
  Recursive:2636.800
Mean Times (s):
 

In [ ]:
import os
import json
import numpy as np
from qiskit import transpile
from disqco.circuits.cp_fraction import cp_fraction
from disqco.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from disqco.parti.FM.FM_main import *
from disqco.parti.FM.multilevel_FM import *

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_MLFM-R_CP_2-12_new.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_MLFM-R_CP_2-12_new.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(16, 97, 8)

# num_partitions_list = [2,4]

fractions = [0.3,0.5,0.7,0.9]
for fraction in fractions:
    for i, num_qubits in enumerate(sizes):

        # Create an All-to-All network
        qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
        network = QuantumNetwork(qpu_info)
        num_partitions = 2 + i
        
        iteration_data = []
        for iteration in range(5):
            
            # -------------------------
            # 1. Define circuit
            # -------------------------

            circuit = cp_fraction(num_qubits,num_qubits,fraction=fraction)

            circuit = transpile(circuit, basis_gates = ['u','cp'])
            base_graph = QuantumCircuitHyperGraph(circuit,group_gates=True)
            depth = base_graph.depth
            initial_assignment = set_initial_partitions(network, num_qubits, depth)
            
            # -------------------------
            # 2. MLFM-R
            # -------------------------
            num_levels = int(np.ceil(np.log2(num_qubits)))
            
            assignment_list_r, cost_list_r, time_list_r = MLFM_recursive(base_graph,
                                        initial_assignment,  
                                        qpu_info, 
                                        limit = num_qubits, 
                                        pass_list= [10]*(num_levels+1), 
                                        stochastic=True, 
                                        lock_nodes=False,
                                        log = False,
                                        add_initial = False,
                                        costs = None)
            
            total_time_r = sum(time_list_r)
            min_cost_r = min(cost_list_r)

            

            
            # -------------------------
            # 3. Store iteration-level results
            # -------------------------
            result_entry = {
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "fraction": fraction,
                "iteration": iteration,
                "r_cost":  min_cost_r,
                "time_r": total_time_r,
            }
            
            detailed_results.append(result_entry)
            iteration_data.append(result_entry)
            
            with open(detailed_filename, "w") as f:
                json.dump(detailed_results, f, indent=2)
        # -------------------------
        # 4. Aggregate results
        r_cost_list = [x["r_cost"] for x in iteration_data]
        r_time_list = [x["time_r"] for x in iteration_data]
        

        mean_r_cost = float(np.mean(r_cost_list))
        mean_r_time = float(np.mean(r_time_list))
        
        print("=============================================")
        print(f"Finished 5 iterations for:")
        print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}")
        print("Mean Costs:")

        print(f"  Recursive:{mean_r_cost:.3f}")
        print("Mean Times (s):")

        print(f"  Recursive:{mean_r_time:.3f}")
        print("=============================================")
        
        mean_entry = {
            "num_qubits": num_qubits,
            "fraction": fraction,
            "num_partitions": num_partitions,
            "mean_r_cost": mean_r_cost,
            "mean_r_time": mean_r_time,
        }
        
        mean_results.append(mean_entry)
        
        with open(means_filename, "w") as f:
            json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Finished 5 iterations for:
  # Qubits: 16, # Partitions: 2
Mean Costs:
  Recursive:6.400
Mean Times (s):
  Recursive:0.036
Finished 5 iterations for:
  # Qubits: 24, # Partitions: 3
Mean Costs:
  Recursive:15.600
Mean Times (s):
  Recursive:0.089
Finished 5 iterations for:
  # Qubits: 32, # Partitions: 4
Mean Costs:
  Recursive:44.400
Mean Times (s):
  Recursive:0.250
Finished 5 iterations for:
  # Qubits: 40, # Partitions: 5
Mean Costs:
  Recursive:86.000
Mean Times (s):
  Recursive:0.574
Finished 5 iterations for:
  # Qubits: 48, # Partitions: 6
Mean Costs:
  Recursive:152.200
Mean Times (s):
  Recursive:1.169
Finished 5 iterations for:
  # Qubits: 56, # Partitions: 7
Mean Costs:
  Recursive:228.600
Mean Times (s):
  Recursive:2.022
Finished 5 iterations for:
  # Qubits: 64, # Partitions: 8
Mean Costs:
  Recursive:316.000
Mean Times (s):
  Recursive:3.242
Finished 5 iterations for:
  # Qubits: 72, # Partitions: 9
Mean Costs:
  Recursive:427.600
Mean Times (s):
  Recursive:4.985
Finis

KeyboardInterrupt: 

In [3]:
import os
import json
import numpy as np
from qiskit import transpile
from qiskit.circuit.library import QFT, QuantumVolume
from disqco.circuits.QAOA import QAOA_random
from disqco.graphs.quantum_network import QuantumNetwork
from disqco.circuits.cp_fraction import cp_fraction
from disqco.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from disqco.parti.FM.FM_main import *
from disqco.parti.FM.multilevel_FM import *

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################

circuit_type = "QAOA"
# circuit_type = "QFT"
# circuit_type = "QV"

detailed_filename = f"benchmark_results_MLFM-R_{circuit_type}_2-4part_new.json"


if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = f"benchmark_means_MLFM-R_{circuit_type}_2-4part_new.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(16, 97, 8)

num_partitions_list = [2,4]

for num_partitions in num_partitions_list:
    for i, num_qubits in enumerate(sizes):

        # Create an All-to-All network
        qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
        network = QuantumNetwork(qpu_info)
        
        iteration_data = []
        for iteration in range(5):
            
            # -------------------------
            # 1. Define/redefine circuit
            # -------------------------
            if circuit_type == "QAOA":
                circuit = QAOA_random(num_qubits,prob=0.5, reps=1)
            elif circuit_type == "QFT":
                circuit = QFT(num_qubits,do_swaps=False)
            elif circuit_type == "QV":
                circuit = QuantumVolume(num_qubits,num_qubits)
            else:
                raise ValueError(f"Unknown circuit type: {circuit_type}")
            
            circuit = transpile(circuit, basis_gates = ['u','cp'])

            base_graph = QuantumCircuitHyperGraph(circuit,group_gates=True)
            depth = base_graph.depth
            initial_assignment = set_initial_partitions(network, num_qubits, depth)
            
            # -------------------------
            # 2. MLFM-R
            # -------------------------
            num_levels = int(np.ceil(np.log2(num_qubits)))
            
            assignment_list_r, cost_list_r, time_list_r = MLFM_recursive(base_graph,
                                        initial_assignment,  
                                        qpu_info, 
                                        limit = num_qubits, 
                                        pass_list= None, 
                                        stochastic=True, 
                                        lock_nodes=False,
                                        log = False,
                                        add_initial = False,
                                        costs = None)
            
            total_time_r = sum(time_list_r)
            min_cost_r = min(cost_list_r)

            # -------------------------
            # 3. Store iteration-level results
            # -------------------------
            result_entry = {
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "iteration": iteration,
                "r_cost":  min_cost_r,
                "time_r": total_time_r,
            }
            
            detailed_results.append(result_entry)
            iteration_data.append(result_entry)
            
            # Update detailed JSON right away
            with open(detailed_filename, "w") as f:
                json.dump(detailed_results, f, indent=2)
        
        # ---------------------------------------------------------------------
        # After 5 iterations, compute the means and log them
        # ---------------------------------------------------------------------

        r_cost_list = [x["r_cost"] for x in iteration_data]
        r_time_list = [x["time_r"] for x in iteration_data]
        
        mean_r_cost = float(np.mean(r_cost_list))
        mean_r_time = float(np.mean(r_time_list))
        
        # Print to console for quick logging
        print("=============================================")
        print(f"Finished 5 iterations for:")
        print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}")
        print("Mean Costs:")

        print(f"  Recursive:{mean_r_cost:.3f}")
        print("Mean Times (s):")

        print(f"  Recursive:{mean_r_time:.3f}")
        print("=============================================")
        
        # Store the aggregated means in a separate JSON
        mean_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "mean_r_cost": mean_r_cost,
            "mean_r_time": mean_r_time,
        }
        
        mean_results.append(mean_entry)
        
        # Update the means JSON file
        with open(means_filename, "w") as f:
            json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Finished 5 iterations for:
  # Qubits: 16, # Partitions: 2
Mean Costs:
  Recursive:7.000
Mean Times (s):
  Recursive:0.219
Finished 5 iterations for:
  # Qubits: 24, # Partitions: 2
Mean Costs:
  Recursive:11.000
Mean Times (s):
  Recursive:0.534
Finished 5 iterations for:
  # Qubits: 32, # Partitions: 2
Mean Costs:
  Recursive:15.000
Mean Times (s):
  Recursive:1.157
Finished 5 iterations for:
  # Qubits: 40, # Partitions: 2
Mean Costs:
  Recursive:19.000
Mean Times (s):
  Recursive:2.129
Finished 5 iterations for:
  # Qubits: 48, # Partitions: 2
Mean Costs:
  Recursive:23.000
Mean Times (s):
  Recursive:3.543
Finished 5 iterations for:
  # Qubits: 56, # Partitions: 2
Mean Costs:
  Recursive:27.000
Mean Times (s):
  Recursive:5.448
Finished 5 iterations for:
  # Qubits: 64, # Partitions: 2
Mean Costs:
  Recursive:31.000
Mean Times (s):
  Recursive:8.206
Finished 5 iterations for:
  # Qubits: 72, # Partitions: 2
Mean Costs:
  Recursive:35.000
Mean Times (s):
  Recursive:11.420
Finished

In [4]:
import os
import json
import numpy as np
from qiskit import transpile
from qiskit.circuit.library import QFT, QuantumVolume
from disqco.circuits.QAOA import QAOA_random
from disqco.graphs.quantum_network import QuantumNetwork
from disqco.circuits.cp_fraction import cp_fraction
from disqco.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from disqco.parti.FM.FM_main import *
from disqco.parti.FM.multilevel_FM import *

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################

# circuit_type = "QAOA"
# circuit_type = "QFT"
circuit_type = "QV"

detailed_filename = f"benchmark_results_MLFM-R_{circuit_type}_2-4part_new.json"


if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = f"benchmark_means_MLFM-R_{circuit_type}_2-4part_new.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(16, 97, 8)

num_partitions_list = [2,4]

for num_partitions in num_partitions_list:
    for i, num_qubits in enumerate(sizes):

        # Create an All-to-All network
        qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
        network = QuantumNetwork(qpu_info)
        
        iteration_data = []
        for iteration in range(5):
            
            # -------------------------
            # 1. Define/redefine circuit
            # -------------------------
            if circuit_type == "QAOA":
                circuit = QAOA_random(num_qubits,prob=0.5, reps=1)
            elif circuit_type == "QFT":
                circuit = QFT(num_qubits,do_swaps=False)
            elif circuit_type == "QV":
                circuit = QuantumVolume(num_qubits,num_qubits)
            else:
                raise ValueError(f"Unknown circuit type: {circuit_type}")
            
            circuit = transpile(circuit, basis_gates = ['u','cp'])

            base_graph = QuantumCircuitHyperGraph(circuit,group_gates=True)
            depth = base_graph.depth
            initial_assignment = set_initial_partitions(network, num_qubits, depth)
            
            # -------------------------
            # 2. MLFM-R
            # -------------------------
            num_levels = int(np.ceil(np.log2(num_qubits)))
            
            assignment_list_r, cost_list_r, time_list_r = MLFM_recursive(base_graph,
                                        initial_assignment,  
                                        qpu_info, 
                                        limit = num_qubits, 
                                        pass_list= None, 
                                        stochastic=True, 
                                        lock_nodes=False,
                                        log = False,
                                        add_initial = False,
                                        costs = None)
            
            total_time_r = sum(time_list_r)
            min_cost_r = min(cost_list_r)

            # -------------------------
            # 3. Store iteration-level results
            # -------------------------
            result_entry = {
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "iteration": iteration,
                "r_cost":  min_cost_r,
                "time_r": total_time_r,
            }
            
            detailed_results.append(result_entry)
            iteration_data.append(result_entry)
            
            # Update detailed JSON right away
            with open(detailed_filename, "w") as f:
                json.dump(detailed_results, f, indent=2)
        
        # ---------------------------------------------------------------------
        # After 5 iterations, compute the means and log them
        # ---------------------------------------------------------------------

        r_cost_list = [x["r_cost"] for x in iteration_data]
        r_time_list = [x["time_r"] for x in iteration_data]
        
        mean_r_cost = float(np.mean(r_cost_list))
        mean_r_time = float(np.mean(r_time_list))
        
        # Print to console for quick logging
        print("=============================================")
        print(f"Finished 5 iterations for:")
        print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}")
        print("Mean Costs:")

        print(f"  Recursive:{mean_r_cost:.3f}")
        print("Mean Times (s):")

        print(f"  Recursive:{mean_r_time:.3f}")
        print("=============================================")
        
        # Store the aggregated means in a separate JSON
        mean_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "mean_r_cost": mean_r_cost,
            "mean_r_time": mean_r_time,
        }
        
        mean_results.append(mean_entry)
        
        # Update the means JSON file
        with open(means_filename, "w") as f:
            json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Finished 5 iterations for:
  # Qubits: 16, # Partitions: 2
Mean Costs:
  Recursive:51.800
Mean Times (s):
  Recursive:0.331
Finished 5 iterations for:
  # Qubits: 24, # Partitions: 2
Mean Costs:
  Recursive:111.000
Mean Times (s):
  Recursive:0.800
Finished 5 iterations for:
  # Qubits: 32, # Partitions: 2
Mean Costs:
  Recursive:206.800
Mean Times (s):
  Recursive:1.506
Finished 5 iterations for:
  # Qubits: 40, # Partitions: 2
Mean Costs:
  Recursive:320.400
Mean Times (s):
  Recursive:2.669
Finished 5 iterations for:
  # Qubits: 48, # Partitions: 2
Mean Costs:
  Recursive:476.000
Mean Times (s):
  Recursive:4.345
Finished 5 iterations for:
  # Qubits: 56, # Partitions: 2
Mean Costs:
  Recursive:624.200
Mean Times (s):
  Recursive:6.493
Finished 5 iterations for:
  # Qubits: 64, # Partitions: 2
Mean Costs:
  Recursive:826.200
Mean Times (s):
  Recursive:9.091
Finished 5 iterations for:
  # Qubits: 72, # Partitions: 2
Mean Costs:
  Recursive:1028.400
Mean Times (s):
  Recursive:13.428

In [5]:
import os
import json
import numpy as np
from qiskit import transpile
from qiskit.circuit.library import QFT, QuantumVolume
from disqco.circuits.QAOA import QAOA_random
from disqco.graphs.quantum_network import QuantumNetwork
from disqco.circuits.cp_fraction import cp_fraction
from disqco.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from disqco.parti.FM.FM_main import *
from disqco.parti.FM.multilevel_FM import *

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################

# circuit_type = "QAOA"
circuit_type = "QFT"
# circuit_type = "QV"

detailed_filename = f"benchmark_results_MLFM-R_{circuit_type}_2-4part_new.json"


if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = f"benchmark_means_MLFM-R_{circuit_type}_2-4part_new.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(16, 97, 8)

num_partitions_list = [2,4]

for num_partitions in num_partitions_list:
    for i, num_qubits in enumerate(sizes):

        # Create an All-to-All network
        qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
        network = QuantumNetwork(qpu_info)
        
        iteration_data = []
        for iteration in range(5):
            
            # -------------------------
            # 1. Define/redefine circuit
            # -------------------------
            if circuit_type == "QAOA":
                circuit = QAOA_random(num_qubits,prob=0.5, reps=1)
            elif circuit_type == "QFT":
                circuit = QFT(num_qubits,do_swaps=False)
            elif circuit_type == "QV":
                circuit = QuantumVolume(num_qubits,num_qubits)
            else:
                raise ValueError(f"Unknown circuit type: {circuit_type}")
            
            circuit = transpile(circuit, basis_gates = ['u','cp'])

            base_graph = QuantumCircuitHyperGraph(circuit,group_gates=True)
            depth = base_graph.depth
            initial_assignment = set_initial_partitions(network, num_qubits, depth)
            
            # -------------------------
            # 2. MLFM-R
            # -------------------------
            num_levels = int(np.ceil(np.log2(num_qubits)))
            
            assignment_list_r, cost_list_r, time_list_r = MLFM_recursive(base_graph,
                                        initial_assignment,  
                                        qpu_info, 
                                        limit = num_qubits, 
                                        pass_list= None, 
                                        stochastic=True, 
                                        lock_nodes=False,
                                        log = False,
                                        add_initial = False,
                                        costs = None)
            
            total_time_r = sum(time_list_r)
            min_cost_r = min(cost_list_r)

            # -------------------------
            # 3. Store iteration-level results
            # -------------------------
            result_entry = {
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "iteration": iteration,
                "r_cost":  min_cost_r,
                "time_r": total_time_r,
            }
            
            detailed_results.append(result_entry)
            iteration_data.append(result_entry)
            
            # Update detailed JSON right away
            with open(detailed_filename, "w") as f:
                json.dump(detailed_results, f, indent=2)
        
        # ---------------------------------------------------------------------
        # After 5 iterations, compute the means and log them
        # ---------------------------------------------------------------------

        r_cost_list = [x["r_cost"] for x in iteration_data]
        r_time_list = [x["time_r"] for x in iteration_data]
        
        mean_r_cost = float(np.mean(r_cost_list))
        mean_r_time = float(np.mean(r_time_list))
        
        # Print to console for quick logging
        print("=============================================")
        print(f"Finished 5 iterations for:")
        print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}")
        print("Mean Costs:")

        print(f"  Recursive:{mean_r_cost:.3f}")
        print("Mean Times (s):")

        print(f"  Recursive:{mean_r_time:.3f}")
        print("=============================================")
        
        # Store the aggregated means in a separate JSON
        mean_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "mean_r_cost": mean_r_cost,
            "mean_r_time": mean_r_time,
        }
        
        mean_results.append(mean_entry)
        
        # Update the means JSON file
        with open(means_filename, "w") as f:
            json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Finished 5 iterations for:
  # Qubits: 16, # Partitions: 2
Mean Costs:
  Recursive:7.000
Mean Times (s):
  Recursive:0.117
Finished 5 iterations for:
  # Qubits: 24, # Partitions: 2
Mean Costs:
  Recursive:11.000
Mean Times (s):
  Recursive:0.350
Finished 5 iterations for:
  # Qubits: 32, # Partitions: 2
Mean Costs:
  Recursive:15.000
Mean Times (s):
  Recursive:0.669
Finished 5 iterations for:
  # Qubits: 40, # Partitions: 2
Mean Costs:
  Recursive:19.000
Mean Times (s):
  Recursive:1.307
Finished 5 iterations for:
  # Qubits: 48, # Partitions: 2
Mean Costs:
  Recursive:23.000
Mean Times (s):
  Recursive:2.041
Finished 5 iterations for:
  # Qubits: 56, # Partitions: 2
Mean Costs:
  Recursive:27.000
Mean Times (s):
  Recursive:3.045
Finished 5 iterations for:
  # Qubits: 64, # Partitions: 2
Mean Costs:
  Recursive:31.000
Mean Times (s):
  Recursive:4.567
Finished 5 iterations for:
  # Qubits: 72, # Partitions: 2
Mean Costs:
  Recursive:35.000
Mean Times (s):
  Recursive:6.451
Finished 